# Program to evolve coupled ODEs using Cash-Karp Runge-Kutta method with adaptive stepwise control

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Define coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    
    # Our equation is d^2y/dx^2 = -y
    # dydx = z
    # dzdx = -y
    
    # we will set y = y[0] and z = y[1]
    
    # declare an array
    y_derivs = np.zeros(2)
    
    # set dydx = z
    y_derivs[0] = y[1]
    
    # set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    # return the array
    return y_derivs

# Define the Cash-Karp method

In [ ]:
def ck_mv_core(dydx,xi,yi,nv,h):
    
    # Declare the k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)    
    k6 = np.zeros(nv)    
    
    # declare temp arrays
    y_temp = np.zeros(nv)
    x_temp = np.zeros(nv)
    
    # get the values of the derivatives at (xi,yi)
    y_derivs = dydx(xi,yi)
    
    # get k1 values
    k1[:] = h * y_derivs[:]
    
    # get k2 values
    x_temp = xi + h*(1./5.)
    y_temp[:] = yi[:] + k1*(1./5.)
    y_derivs = dydx(x_temp,y_temp)
    k2[:] = h * y_derivs[:]
    
    # get k3 values
    x_temp = xi + h*(3./10.)
    y_temp[:] = yi[:] + (k1 * 3./40.) + (k2 * 9./40.)
    y_derivs = dydx(x_temp,y_temp)
    k3[:] = h * y_derivs[:]
    
    # get k4 values
    x_temp = xi + h*(3./5.)
    y_temp[:] = yi[:] + (k1 * 3./10.) + (k2 * -9./10.) + (k3 * 6./5.)
    y_derivs = dydx(x_temp,y_temp)
    k4[:] = h * y_derivs[:]
    
    # get k5 values
    x_temp = xi + h
    y_temp[:] = yi[:] + (k1 * -11./54.) + (k2 * 5./2.) + (k3 * -70./27.) + (k4 * 35./27.)
    y_derivs = dydx(x_temp,y_temp)
    k5[:] = h * y_derivs[:]
    
    # get k6 values
    x_temp = xi + h*(7./8.)
    y_temp[:] = yi[:] + (k1 * 1631./55296.) + (k2 * 175./512.) + (k3 * 575./13824.) + (k4 * 44275./110592.) + (k5 * 253./4096.)
    y_derivs = dydx(x_temp,y_temp)
    k6[:] = h * y_derivs[:]
    
    
    # advance y by a step h
    y_ipo = yi + (k1 * 37./378.) + (k3 * 250./621.) + (k4 * 125./594.) + (k6 * 512./1771.)
    y_ipostar = yi + (k1 * 2825./27648.) + (k3 * 18575./48384.) + (k4 * 13525./55296.) + (k5 * 277./14336.) + (k6 * 1./4.)
    
    
    return y_ipo, y_ipostar

# Define an adaptive step size driver for Cash-Karp

In [ ]:
def ck_mv_ad(dydx,x_i,y_i, nv,h,tol):
    
    # define the safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    # set a maximum number of iterations
    imax = 10000
    
    # set an iteration variable
    i = 0
    
    # create an error array
    Delta = np.full(nv,2*tol)
    
    # remember the step
    h_step = h
    
    # adjust the step
    while(Delta.max()/tol > 1.0):
        
        # find error by taking a step
        y_ipo, y_ipostar = ck_mv_core(dydx,x_i,y_i,nv,h_step)
        
        # compute the error
        Delta = np.fabs(y_ipo - y_ipostar)
        
        # if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            # decrease step size
            h_step *= (SAFETY * (Delta.max()/tol)**(-0.25))
        
        # check for too many iterations
        if(i>=imax):
            print("Too many iterations in ck_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        
        # iterate loop
        i+=1
    
    # next time try a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)

    
    # return the answer, new step, step we took
    return y_ipo, h_new, h_step

# Define a wrapper for Cash Karp

In [ ]:
def ck_mv(dfdx,a,b,y_a,tol):
    # dydx is derivative wrt x
    # a is the lower bound
    # b is the upper bound
    # y_a are the boundary conditions
    # tol is the tolerance for integrating y
    
    # define our starting step
    xi = a
    yi = y_a.copy()
    
    # set the intial step size to be very small
    h = 1.0e-4 * (b-a)
    
    # set the max number of iterations
    imax = 10000
    
    # set iteration var
    i = 0
    
    # set the number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    # set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    
    flag = 1
    
    # loop until we reach the right side
    while(flag):
        
        # calculate y_i+1
        yi_new, h_new, h_step = ck_mv_ad(dydx,xi,yi,nv,h,tol)
        
        # update the step
        h = h_new
        
        # prevent an overshoot
        if(xi+h_step>b):
            
            # take a smaller step
            h = b-xi
            
            # recalculate y_i+1
            yi_new, h_new, h_step = ck_mv_ad(dydx,xi,yi,nv,h,tol)
            
            # break out of loop
            flag = 0
        
        # update values
        xi += h_step
        yi[:] = yi_new[:]
        
        # add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        # prevent too many iterations
        if (i>=imax):
            
            print("Too many iterations in rk4_mv()")
            raise StopIteration("Ending after i = ",i)
        
        # iterate
        i += 1
        
        # output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb = %9.8f" % (i,xi,h_step,b)
        print(s)
        
        # break if new xi is == b
        if(xi==b):
            flag = 0
            
    return x,y

# Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

# perform the integration
x,y = ck_mv(dydx,a,b,y_0,tolerance)

# Plot the result

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

# Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)